In [ ]:
# from google.colab import drive
# try:
#     from google.colab import drive
#     drive.mount('/content/drive')
#     workspace = '/content/drive/MyDrive'
# except:
#     workspace = '.'
!pip install torch==1.7.0 torchvision==0.8.0 torchaudio==0.7.0
%pip install -U opencv-python
%pip install -U opencv-contrib-python
# %pip install -U grad-cam
%pip install eniops
%pip install vit-pytorch
%pip install -U grad-cam

#Read and load the images

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
labels={"real":0,"cyberpunk":1,"cartoon":2,"No-style":3}
category={0:"real",1:"cyberpunk",2:"cartoon",3:"No-style"}
# Path to the folder containing the images
folder_path = 'Real_images/Real_images'
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

inference_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

def img_transform(img_rgb, transform=None):
    """
    transform images
    :param img_rgb: PIL Image
    :param transform: torchvision.transform
    :return: tensor
    """

    if transform is None:
        raise ValueError("there is no transform")

    img_t = transform(Image.fromarray(img_rgb))
    return img_t

class Sample:
    def __init__(self, idx=0, fname='', img=None, feat=None, label=None):
        self.idx = idx
        self.fname = fname
        self.img = img
        self.torch_img=img
        self.vgg_feat = feat
        self.label = label
        self.patches = None
        self.pred_set = []
        self.si_score = []
        self.pred = None
# fig, ax = plt.subplots()
samples=[]
image_files = [file for file in os.listdir(folder_path) if file.endswith(('.jpg', '.jpeg', '.png'))]
print(len(image_files))
for i, file in enumerate(image_files):
  image_path = os.path.join(folder_path, file)
  img = cv2.imread(image_path, cv2.IMREAD_COLOR)[..., ::-1]
  if i == 0:
    H, W, C = img.shape
  else:
    img = cv2.resize(img, (W, H))
    # plt.title(file)
    # plt.imshow(img)
  samples.append(Sample(img=img,label=0))
print(len(samples))
folder_path = 'Real_images/Indoor'
image_files = [file for file in os.listdir(folder_path) if file.endswith(('.jpg', '.jpeg', '.png'))]
for i, file in enumerate(image_files):
  image_path = os.path.join(folder_path, file)
  img = cv2.imread(image_path, cv2.IMREAD_COLOR)[..., ::-1]
  img = cv2.resize(img, (W, H))
    # plt.title(file)
    # plt.imshow(img)
  samples.append(Sample(img=img,label=0))
print(len(samples))

folder_path='AI_generated_images/AI_generated_images'
folder_paths = ['/cyberpunk', '/Cartoon', '/No_style']
for i in range(3):
  folder_paths[i]=folder_path+folder_paths[i]
# Iterate over the folder paths
for i, folder_path in enumerate(folder_paths):
    # Get a list of all image files in the folder
  image_files = [file for file in os.listdir(folder_path) if file.endswith(('.jpg', '.jpeg', '.png'))]

  for j, file in enumerate(image_files):
    image_path=os.path.join(folder_path,file)
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)[..., ::-1]
    img = cv2.resize(img, (W, H))
    # plt.imshow(img)
    samples.append(Sample(img=img,label=i+1))
  print(np.array(samples).shape)
# Adjust the spacing between subplots
# plt.tight_layout()

In [ ]:
for sample in samples:
  sample.torch_img=img_transform(sample.img, inference_transform)

# Read the test dataset

The reference dataset

In [ ]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(samples, test_size=0.2, random_state=42)

In [ ]:
folder_path='references'
folder_paths = ['/AI',"/Real" ]
test=[]
for i in range(2):
  folder_paths[i]=folder_path+folder_paths[i]
# Iterate over the folder paths
for i, folder_path in enumerate(folder_paths):
    # Get a list of all image files in the folder
  image_files = [file for file in os.listdir(folder_path) if file.endswith(('.jpg', '.jpeg', '.png'))]

  for j, file in enumerate(image_files):
    image_path=os.path.join(folder_path,file)
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)[..., ::-1]
    img = cv2.resize(img, (W, H))
    # plt.imshow(img)
    if("AI" in folder_paths[i]):
        sample=Sample(img=img,label=3)
        samples.append(sample)
        test.append(sample)
    else:
        sample=Sample(img=img,label=0)
        samples.append(sample)
        test.append(sample)
  print(np.array(samples).shape)

In [ ]:
train[3].torch_img.shape

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(16, 6))

def color_histogram(img : np.array, norm : bool = True) -> np.array:
    """ Count the multi-channel histogram

    Params:
        img:
            An image with size of H x W x C.
        norm:
            Whether normalize the sum of the histogram for each channel to 1.
    """
    hists=[]
    for i in range(3):
      hists.append(np.array([len(img[...,i][img[...,i]==k]) for k in range(256)])/np.size(img))
    return np.array(hists)
# axs.figure(figsize=(10, 3))
axs[0,0].set_title('Real')
axs[0,0].imshow(validation[0].img)
axs[0, 0].axis('off')

hists = color_histogram(validation[0].img)
display(type(hists[0]))
# ##TODO for plot
# axs[0,1].set_title('RGB')
axs[0,1].plot(range(256),hists[0],c='r')
axs[0,1].plot(range(256),hists[1],c='g')
axs[0,1].plot(range(256),hists[2],c='b')
axs[1,0].imshow(validation[1].img)
axs[1, 0].axis('off')
axs[0,2].set_title('Gen-Cartoon')
axs[0,2].imshow(validation[5].img)
axs[0,2].axis('off')
# axs[0,3].set_title('histogram (RGB)')
hists = color_histogram(validation[5].img)
display(type(hists[0]))
# ##TODO for plot
axs[0,3].plot(range(256),hists[0],c='r')
axs[0,3].plot(range(256),hists[1],c='g')
axs[0,3].plot(range(256),hists[2],c='b')
axs[1,0].set_title('Gen-No Style')
axs[1,0].imshow(validation[1].img)
axs[1, 0].axis('off')
# axs[1,1].set_title('histogram (RGB)')
hists = color_histogram(validation[1].img)
display(type(hists[0]))
# ##TODO for plot
axs[1,1].plot(range(256),hists[0],c='r')
axs[1,1].plot(range(256),hists[1],c='g')
axs[1,1].plot(range(256),hists[2],c='b')
axs[1,2].set_title('Gen-Cyberpunk')
axs[1,2].imshow(validation[11].img)
axs[1, 2].axis('off')
# axs[1,3].set_title('histogram (RGB)')
hists = color_histogram(validation[11].img)
display(type(hists[0]))
# ##TODO for plot
axs[1,3].plot(range(256),hists[0],c='r')
axs[1,3].plot(range(256),hists[1],c='g')
axs[1,3].plot(range(256),hists[2],c='b')

# Try to build test the model based on Simple NN

In [ ]:
train_feats = [sample.torch_img.numpy() for sample in train]
train_labels = [sample.label for sample in train]
validation_feats = [sample.torch_img.numpy() for sample in validation]
validation_labels = [sample.label for sample in validation]
test_feats=[sample.torch_img.numpy() for sample in test]
test_labels=[sample.label for sample in test]

In [ ]:
num_cats=4
temp_cats = [[0] * num_cats for _ in range(len(train_labels))]
for i in range(len(train_labels)):
  temp_cats[i][train_labels[i]]=1
train_cats=temp_cats
temp_cats = [[0] * num_cats for _ in range(len(validation_labels))]
for i in range(len(validation_labels)):
  temp_cats[i][validation_labels[i]]=1
val_cats=temp_cats
temp_cats = [[0] * num_cats for _ in range(len(test_labels))]
for i in range(len(test_labels)):
  temp_cats[i][test_labels[i]]=1
test_cats=temp_cats

In [ ]:
from torch.functional import Tensor
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

train_feats = torch.tensor(train_feats).to(torch.float32)
train_x = torch.reshape(train_feats,[train_feats.shape[0],3,256,341])
train_y = torch.tensor(train_cats).to(torch.float32)
val_feats = torch.tensor(validation_feats).to(torch.float32)
val_x = torch.reshape(val_feats,[val_feats.shape[0],3,256,341])
val_y = torch.tensor(val_cats).to(torch.float32)
# test_feats = torch.tensor(test_feats).to(torch.float32)
# test_x = torch.reshape(test_feats,[test_feats.shape[0],3,256,341])
# test_y = torch.tensor(test_cats).to(torch.float32)
# Specify batch size
batch_size = 32
num_batches = len(train_x) // batch_size
num_epochs=200
output_channel=4
class SIMPLENN(nn.Module):
    def __init__(self):
        super(SIMPLENN, self).__init__()
        #because the shape of the image is gray image, so the input chaneel should be 1
        self.conv1 = nn.Conv2d(3,6,kernel_size=5)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(6,16, kernel_size=5)
        self.pool2 = nn.MaxPool2d(kernel_size=2,stride=2)
        self.linear1 = nn.Linear(80032, 4096)
        self.drop1 = nn.Dropout(0.2)
        self.linear2 = nn.Linear(4096,200)
        self.linear3 = nn.Linear(200,4)
    # forward function is inherted from parent's class. x denotes the input feature.
    def forward(self, x):
        y_pred = self.conv1(x)
        y_pred = F.relu(y_pred)
        y_pred = self.pool1(y_pred)
        y_pred = F.relu(self.conv2(y_pred))
        y_pred = self.pool2(y_pred)
        y_pred = y_pred.view(-1,80032)

        y_pred = F.relu(self.linear1(y_pred))
        y_pred = self.linear2(y_pred)
        y_pred = self.drop1(y_pred)
        y_pred = self.linear3(F.relu(y_pred))
        return y_pred
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')
# create model
model = SIMPLENN().to(device)

criterion = nn.CrossEntropyLoss()

# create optimizer. 1st parameter: the parameters will be optimized; 2nd: learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
train_conv=[]
# Define the training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx + 1) * batch_size

        if torch.cuda.is_available():
            inputs = Variable(train_x[start_idx:end_idx]).cuda()
            target = Variable(train_y[start_idx:end_idx]).cuda()
        else:
            inputs = Variable(train_x[start_idx:end_idx])
            target = Variable(train_y[start_idx:end_idx])
                # Clear gradients
        optimizer.zero_grad()
        # Forward pass
        out= model(inputs)
        # Calculate loss
        loss = criterion(out, target)

        # Backward propagation
        loss.backward()
        # Updating parameters via SGD
        optimizer.step()
        # Print training progress
        if(batch_idx==num_batches-1):
          print(f"Batch {batch_idx+1}/{num_batches} - Loss: {loss.item()}")
    if(loss<0.001):
        break

In [ ]:
out=model(val_x)
from sklearn.metrics import accuracy_score

max_values = np.argmax(out.detach().cpu().numpy(), axis=1)
true_labels=[]
for label in validation_labels:
    if(label>1):
        true_labels.append(1)
    else:
        true_labels.append(0)
pred_labels=[]
for label in max_values:
    if(label>1):
        pred_labels.append(1)
    else:
        pred_labels.append(0)
# accuracy = accuracy_score(pred_labels, true_labels)
accuracy = accuracy_score(max_values, validation_labels)
accuracy

In [ ]:
out=model(test_x)
from sklearn.metrics import accuracy_score

max_values = np.argmax(out.detach().cpu().numpy(), axis=1)
true_labels=[]
for label in test_labels:
    if(label>1):
        true_labels.append(1)
    else:
        true_labels.append(0)
pred_labels=[]
for label in max_values:
    if(label>1):
        pred_labels.append(1)
    else:
        pred_labels.append(0)
# accuracy = accuracy_score(pred_labels, true_labels)
accuracy = accuracy_score(max_values, test_labels)
accuracy

# Try to extract features by VGG16 net

In [ ]:
len(train_feats)

In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import models as models
from torch.autograd import Variable
train_feats = torch.tensor(train_feats).to(torch.float32)
train_x = torch.reshape(train_feats,[train_feats.shape[0],3,256,341])
val_feats = torch.tensor(validation_feats).to(torch.float32)
val_x = torch.reshape(val_feats,[val_feats.shape[0],3,256,341])
train_y = torch.tensor(train_cats).to(torch.float32)
val_y = torch.tensor(val_cats).to(torch.float32)
# load a pretrained DCNN (e.g., VGG)
class VGGFeature(nn.Module):
    def __init__(self, pretrained=True, layer=28):
        super().__init__()
        self.net = models.vgg16(pretrained).features.eval()
        self.layer = layer

        self.layer1 = nn.Linear(512*21*16, 512)
        self.layer2 = nn.Linear(512,64)
        self.layer3 = nn.Linear(64,4)
        self.drop1=nn.Dropout(0.2)
    def forward(self, x):
      for idx, layer in enumerate(self.net):
          x = layer(x)
          if idx == self.layer:
              break
      y_pred=x.view(-1,512*21*16)
      y_pred=F.relu(y_pred)
      y_pred=self.layer1(y_pred)
      y_pred=F.relu(y_pred)
      train_conv=y_pred
      y_pred=self.layer2(y_pred)
      # y_pred=self.drop1(y_pred)
      y_pred=F.relu(y_pred)
      y_pred=self.layer3(y_pred)
      return y_pred,train_conv
force_cpu = False

if torch.cuda.is_available() and not force_cpu:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print('We are using device', device)

VGG = VGGFeature().to(device)
criterion = nn.CrossEntropyLoss()

# create optimizer. 1st parameter: the parameters will be optimized; 2nd: learning rate
optimizer = torch.optim.SGD(VGG.parameters(), lr=0.05)
train_conv=[]
batch_size = 16
num_batches = len(train_feats) // batch_size
num_epochs=50
train_loss_values = []
val_loss_values = []
best_loss=100
# Define the training loop
counter=0
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    running_train_loss = 0.0
    for batch_idx in range(num_batches):
        start_idx = batch_idx * batch_size
        end_idx = (batch_idx + 1) * batch_size

        if torch.cuda.is_available():
            inputs = Variable(train_x[start_idx:end_idx]).cuda()
            target = Variable(train_y[start_idx:end_idx]).cuda()
        else:
            inputs = Variable(train_x[start_idx:end_idx])
            target = Variable(train_y[start_idx:end_idx])
                # Clear gradients
        optimizer.zero_grad()
        # Forward pass
        out,_= VGG(inputs)
        # Calculate loss
        loss = criterion(out, target)

        # Backward propagation
        loss.backward()
        # Updating parameters via SGD
        optimizer.step()
        # Print training progress
        if(batch_idx==num_batches-1):
          print(f"Batch {batch_idx+1}/{num_batches} - Loss: {loss.item()}")
        running_train_loss += loss.item()
    epoch_train_loss = running_train_loss / train_x.shape[0]
    train_loss_values.append(epoch_train_loss)
    # Validation phase
    VGG.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        val_outputs,_ = VGG(val_x)
        val_loss = criterion(val_outputs, val_y)
        running_val_loss += val_loss.item()
    len_val=15
    # # Calculate average validation loss for the epoch
    epoch_val_loss = running_val_loss / len_val
    val_loss_values.append(epoch_val_loss)
    if(running_val_loss<best_loss):
        best_loss=running_val_loss
        best_model=VGG
        counter=0
    else:
        counter+=1
    if(counter>5):
        break
    # Print the loss for the epoch
    print(f"Epoch {epoch+1} - Training Loss: {epoch_train_loss}, Validation Loss: {epoch_val_loss}")

# Plot the loss values for training and validation sets on the same chart
plt.plot(train_loss_values, marker='o', label='Training Loss')
plt.plot(val_loss_values, marker='o', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss during Training and Validation')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
train_out,train_conv=best_model(train_x)
val_out,val_conv = best_model(val_x)
test_out,test_conv = best_model(test_x)

In [ ]:
# Plot the loss values for training and validation sets on the same chart
plt.plot(train_loss_values, marker='', label='Training Loss')
plt.plot(val_loss_values, marker='', label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss during Training and Validation')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
out,_= best_model(test_x)
max_values = np.argmax(out.detach().numpy(), axis=1)
true_labels=[]
for label in test_labels:
    if(label>1):
        true_labels.append(1)
    else:
        true_labels.append(0)
pred_labels=[]
for label in max_values:
    if(label>1):
        pred_labels.append(1)
    else:
        pred_labels.append(0)
# accuracy = accuracy_score(pred_labels, true_labels)
accuracy = accuracy_score(max_values, test_labels)
accuracy

SVM + VGG

In [ ]:
train_conv=train_conv.detach().cpu().requires_grad_(False)
ind=0
for conv in train_conv.numpy():
  train[ind].vgg_feat=conv
  ind+=1
val_conv=val_conv.detach().cpu()
ind=0
for conv in val_conv.numpy():
  validation[ind].vgg_feat=conv
  ind+=1
ind=0
test_conv=test_conv.detach().cpu().requires_grad_(False)
for conv in test_conv.numpy():
  test[ind].vgg_feat=conv
  ind+=1

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score
train_feat=[sample.vgg_feat for sample in train]
train_label=[sample.label for sample in train]
val_feat=[sample.vgg_feat for sample in validation]
val_label=[sample.label for sample in validation]
test_feat=[sample.vgg_feat for sample in test]
test_label=[sample.label for sample in test]
# 3. Model Training
from sklearn.ensemble import RandomForestClassifier
# svm_classifier = svm.SVC(kernel='linear')
svm_classifier = RandomForestClassifier()
svm_classifier.fit(train_feat, train_label)

# 4. Model Evaluation
y_pred = svm_classifier.predict(val_feat)
accuracy = accuracy_score(val_label, y_pred)
print("Accuracy:", accuracy)

# # 5. Prediction
# # Assuming you have a new, unseen image represented by its features stored in a variable called 'new_image_features'
predicted_class = svm_classifier.predict(val_feat)
print("Predicted Class:", predicted_class)
print("True class",val_label)

# 4. Model test
y_pred = svm_classifier.predict(test_feat)
accuracy = accuracy_score(test_label, y_pred)
print("Test Accuracy:", accuracy)

# # 5. Prediction
# # Assuming you have a new, unseen image represented by its features stored in a variable called 'new_image_features'
predicted_class = svm_classifier.predict(test_feat)
print("Test Predicted Class:", predicted_class)
print("Test True class",test_label)

In [ ]:
i=0
correct=0
for result in test_label:
    if(result==0):
        if(predicted_class[i]==0):
            correct+=1
    elif(result!=0):
        if(predicted_class[i]!=0):
            correct+=1
        i+=1

In [ ]:
correct/len(test_label)

# VGG Feature extract + Patchify + Siamonse network

In [ ]:
import numpy as np

def patchify_image(image, patch_size):
    '''
    Patchify a colorful image into non-overlapping patches.

    Parameters:
        image (numpy.ndarray): Input image as a NumPy array.
        patch_size (tuple): Size of the patches (height, width).

    Returns:
        numpy.ndarray: Patchified image as a 4D array with shape (num_patches, patch_height, patch_width, num_channels).
    '''
    height, width, channels = image.shape
    patch_height, patch_width = patch_size

    num_patches_h = height // patch_height
    num_patches_w = width // patch_width
    num_patches = num_patches_h * num_patches_w

    patchified_image = np.zeros((num_patches, patch_height, patch_width, channels), dtype=image.dtype)

    index = 0
    for h in range(num_patches_h):
        for w in range(num_patches_w):
            patch = image[h*patch_height:(h+1)*patch_height, w*patch_width:(w+1)*patch_width, :]
            patchified_image[index] = patch
            index += 1

    return patchified_image


# Load the colorful image
image = train[1].img

# Define patch size and overlap
# Convert image to RGB if needed
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Define patch size
patch_size = (1008,1344)

# Patchify the image
patchified_image = patchify_image(image, patch_size)

# Display different patches
num_patches_to_show = 9

# Create a 4x4 subplot grid
fig, axes = plt.subplots(3, 3, figsize=(16, 16))

# Iterate over the patchified images and display them in the grid
for i, ax in enumerate(axes.flat):
    ax.imshow(patchified_image[i])
    ax.axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the grid of patchified images
plt.show()

Create the set for patches

In [ ]:
class SubSample:
    def __init__(self, idx=0, fname='', img=None, feat=None, label=None):
        self.idx = idx
        self.feat=feat
        self.img = img
#         self.torch_img=img
        self.label = label
        self.pred_set = []
        self.si_score = []
        self.pred = None
import gc
i=0
patch_train=[]
for sample in train:
    sample.patches=patchify_image(sample.img, (1008,1344))
    torch_patch=[]
    for patch in sample.patches:
#         print(img_transform(patch, inference_transform).shape)
        _,conv=best_model(img_transform(patch, inference_transform))
        torch_patch.append(torch.squeeze(conv).detach().requires_grad_(False))
        patch_train.append(SubSample(idx=i,img=patch,feat=torch_patch[-1],label=sample.label))
    gc.collect()
    sample.patches=torch_patch
#     print(i)
    i+=1

In [ ]:
patch_val=[]
for sample in validation:
    sample.patches=patchify_image(sample.img, (1008,1344))
    torch_patch=[]
    for patch in sample.patches:
#         print(img_transform(patch, inference_transform).shape)
        _,conv=best_model(img_transform(patch, inference_transform))
        torch_patch.append(torch.squeeze(conv).detach().requires_grad_(False))
        patch_val.append(SubSample(idx=i,img=patch,feat=torch_patch[-1],label=sample.label))
    gc.collect()
    sample.patches=torch_patch
#     print(i)
    i+=1

In [ ]:
patch_test=[]
for sample in test:
    sample.patches=patchify_image(sample.img, (1008,1344))
    torch_patch=[]
    for patch in sample.patches:
#         print(img_transform(patch, inference_transform).shape)
        _,conv=best_model(img_transform(patch, inference_transform))
        torch_patch.append(torch.squeeze(conv).detach().requires_grad_(False))
        patch_test.append(SubSample(idx=i,img=patch,feat=torch_patch[-1],label=sample.label))
    gc.collect()
    sample.patches=torch_patch
#     print(i)
    i+=1

In [ ]:
import gc
i=0
for sample in samples:
    sample.patches=patchify_image(sample.img, (1008,1344))
    torch_patch=[]
    for patch in sample.patches:
#         print(img_transform(patch, inference_transform).shape)
        _,conv=best_model(img_transform(patch, inference_transform))
        torch_patch.append(torch.squeeze(conv).detach().requires_grad_(False))
    gc.collect()
    sample.patches=torch_patch
#     print(i)
    i+=1
    

# ** Train for Siamese

In [ ]:
train_feats = torch.empty(0, 512)
train_cats =[]

for sample in train:
    stacked = torch.stack(sample.patches, dim=0)
#     print(stacked.shape)
    train_feats = torch.cat([stacked, train_feats], dim=0)
    cat=[sample.label for c in range(9)]
    train_cats.append(cat)
train_cats=torch.from_numpy(np.array(train_cats).flatten())
# Generate random permutation indices
indices = torch.randperm(len(train_cats))

# Shuffle both tensor arrays using the generated indices
train_feats = train_feats[indices]
train_cats = train_cats[indices]

In [ ]:
train[0].patches[0].shape

In [ ]:
from collections import OrderedDict
# build the networks
class SiameseNet(nn.Module):
    def __init__(self, in_features=512, mid_features=256, out_features=128):
        super().__init__()
        self.net = nn.Sequential(OrderedDict([
            ('Input', nn.Linear(in_features, out_features)),
#             ('Act', nn.ReLU()),
#             ('Output', nn.Linear(mid_features, out_features)),
        ]))
        self.cos_sim = nn.CosineSimilarity(dim=1)

    def forward(self, x, y):
        feat_x = self.net(x)
        feat_y = self.net(y)
#         print(feat_x.shape)
#         print(self.cos_sim(feat_x, feat_y))
        return self.cos_sim(feat_x, feat_y)

Net = SiameseNet().to(device)
optimizer = torch.optim.Adam(Net.parameters(), lr=1e-4, betas=(0.9, 0.999))
criterion = nn.MSELoss()

num_iters = 100000
batch_size = 32
n_train = len(train_feats)
# train_inputs = torch.cat([sample. for sample in train], dim=0).to(device)
# train_labels = torch.tensor([sample.label for sample in train]).to(device)

# print(train_inputs.shape, train_labels.shape)
# training
best_loss=100
for it in range(num_iters):
    idx_x = torch.randint(n_train, size=(batch_size,), device=device)
    idx_y = torch.randint(n_train, size=(batch_size,), device=device)
    
    input_x = train_feats[idx_x]
    input_y = train_feats[idx_y]
    target = (train_cats[idx_x] == train_cats[idx_y]).to(torch.float32) * 2 - 1
    output = Net(input_x, input_y)
    # print(output, target)
    loss = criterion(output, target)
    if(loss<best_loss):
        best_loss=loss
        best_Net=Net
    if it % 10000 == 0:
        print(loss.item())
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# Build histogram for whole image(Siamese)

In [ ]:
i=0
for sample in train:
    sample.idx=i
    sample.si_score=np.zeros(len(train))
    i+=1
for sample in validation:
    sample.idx=i
    sample.si_score=np.zeros(len(train))
    i+=1

In [ ]:
all_feats = torch.cat([feat.view(1, -1) for feat in train_feats], dim=0)
best_Net.eval()
for idx, sample in enumerate(patch_train):
    dists =  best_Net(sample.feat.view(1, -1).expand_as(all_feats), all_feats)
    simlarity, orders = torch.sort(dists, descending=True)

#     plt.figure(figsize=(18,4))
#     plt.subplot(1, 6, 1)
#     plt.title(f'query:label:{category[sample.label]} idx{sample.idx}')
#     plt.imshow(sample.img)
    
    for i, order in enumerate(orders[1:6]):
#         plt.subplot(1, 6, i+2)
        result = patch_train[order]
        train[sample.idx].si_score[result.idx]+=dists[order]
#         plt.title(f'mark:{dists[order]}')
#         plt.imshow(result.img)
#     if(idx>5):
#         break
for idx, sample in enumerate(patch_val):
    dists =  best_Net(sample.feat.view(1, -1).expand_as(all_feats), all_feats)
    simlarity, orders = torch.sort(dists, descending=True)

#     plt.figure(figsize=(18,4))
#     plt.subplot(1, 6, 1)
#     plt.title(f'query:label:{category[sample.label]} idx{sample.idx}')
#     plt.imshow(sample.img)
    
    for i, order in enumerate(orders[1:6]):
#         plt.subplot(1, 6, i+2)
        result = patch_train[order]
        validation[sample.idx-len(train)].si_score[result.idx]+=dists[order]
#         plt.title(f'label{category[sample.label]}mark:{dists[order]}')
#         plt.imshow(result.img)
#     if(idx>5):
#         break

In [ ]:
label=[]
for idx in range(len(validation[0].si_score)):
    label.append(train[idx].label)

In [ ]:
print(*label)

In [ ]:
color=[]
# Example data
x_values = label
y_values = validation[0].si_score
k=0
for value in x_values:
    if(value==0):
        k+=1
        color.append("orange")
    if(value==1):
        color.append("black")
    if(value==2):
        color.append("purple")
    if(value==3):
        color.append("yellow")
# Generate x-axis tick positions
# x_tick_positions = range(len(x_values))

# # Sort x_values and update y_values accordingly
# sorted_data = sorted(zip(x_values, y_values))
# x_values, y_values = zip(*sorted_data)

# Generate x-axis tick positions
x_tick_positions = range(len(x_values))

# Plot the bars
plt.bar(x_tick_positions, y_values,color=color)

# Set x-axis tick positions and labels
plt.xticks(x_tick_positions, x_values)

# Set x-axis and y-axis labels
plt.xlabel('Similarity')
plt.ylabel('Categories')

# Show the plot
plt.show()

To classify different patches independently

In [ ]:
i=0
for sample in train:
    sample.idx=i
    sample.si_score=np.zeros((9,len(train)))
    i+=1
for sample in validation:
    sample.idx=i
    sample.si_score=np.zeros((9,len(train)))
    i+=1

In [ ]:
all_feats = torch.cat([feat.view(1, -1) for feat in train_feats], dim=0)
best_Net.eval()
p=0
for idx, sample in enumerate(patch_train):
    dists =  best_Net(sample.feat.view(1, -1).expand_as(all_feats), all_feats)
    simlarity, orders = torch.sort(dists, descending=True)

#     plt.figure(figsize=(18,4))
#     plt.subplot(1, 6, 1)
#     plt.title(f'query:label:{category[sample.label]} idx{sample.idx}')
#     plt.imshow(sample.img)
    if(p==9):
        p=0
    for i, order in enumerate(orders[1:56]):
#         plt.subplot(1, 6, i+2)
        result = patch_train[order]
        train[sample.idx].si_score[p][result.idx]+=1
    p+=1
#         plt.title(f'mark:{dists[order]}')
#         plt.imshow(result.img)
#     if(idx>5):
#         break
p=0
for idx, sample in enumerate(patch_val):
    dists =  best_Net(sample.feat.view(1, -1).expand_as(all_feats), all_feats)
    simlarity, orders = torch.sort(dists, descending=True)

#     plt.figure(figsize=(18,4))
#     plt.subplot(1, 6, 1)
#     plt.title(f'query:label:{category[sample.label]} idx{sample.idx}')
#     plt.imshow(sample.img)
    if(p==9):
        p=0
    for i, order in enumerate(orders[1:56]):
#         plt.subplot(1, 6, i+2)
        result = patch_train[order]
        validation[sample.idx-len(train)].si_score[p][result.idx]+=1
    p+=1
#         plt.title(f'label{category[sample.label]}mark:{dists[order]}')
#         plt.imshow(result.img)
#     if(idx>5):
#         break

# Prepare train and val feats(with SVM and RFC classifier, Siamese)

In [ ]:
si_train_feat=train[0].si_score
si_train_label=[]
si_train_label.append([train[0].label for i in range(9)])
for sample in train[1:]:
    si_train_feat=np.vstack((si_train_feat,sample.si_score))
    si_train_label.append([sample.label for i in range(9)])

In [ ]:
si_train_label=np.array(si_train_label).reshape(9*len(train))

In [ ]:
si_val_feat=validation[0].si_score
si_val_label=[]
si_val_label.append([validation[0].label for i in range(9)])
for sample in validation[1:]:
    si_val_feat=np.vstack((si_val_feat,sample.si_score))
    si_val_label.append([sample.label for i in range(9)])

In [ ]:
si_val_label=np.array(si_val_label).reshape(9*len(validation))

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
# Create an instance of SVM classifier
# classifier = SVC(kernel='poly',degree=3)
classifier = RandomForestClassifier()
# Train the classifier
classifier.fit(si_train_feat, si_train_label)

predictions = classifier.predict(si_val_feat)
print("Predictions:", predictions)
accuracy = accuracy_score(si_val_label, predictions)
print("Accuracy:", accuracy)

In [ ]:
vote=predictions.reshape(15,9)

index=0
for result in vote:
    real=0
    nostyle=0
    cyberpunk=0
    cartoon=0
    for expert in result:
        if (expert==0):
            real+=1
        elif(expert==1):
            cyberpunk+=1
        elif(expert==2):
            cartoon+=1
        else:
            nostyle+=1
    if(real>=5):
        validation[index].pred=0
    else:
        if cyberpunk >= cartoon and cyberpunk >= nostyle:
            validation[index].pred=1
        elif cartoon >= cyberpunk and cartoon >= nostyle:
            validation[index].pred=2
        else:
            validation[index].pred=3
    index+=1


In [ ]:
pred=[sample.pred for sample in validation]
true=[sample.label for sample in validation]
accuracy = accuracy_score(pred, true)
print("Accuracy:", accuracy)

# Hard voting(Siamese)

In [ ]:
i=0
for sample in train:
    sample.idx=i
    sample.si_score=np.zeros((9,4))
    i+=1
for sample in validation:
    sample.idx=i
    sample.si_score=np.zeros((9,4))
    i+=1

In [ ]:
all_feats = torch.cat([feat.view(1, -1) for feat in train_feats], dim=0)
best_Net.eval()
p=0
for idx, sample in enumerate(patch_train):
    dists =  best_Net(sample.feat.view(1, -1).expand_as(all_feats), all_feats)
    simlarity, orders = torch.sort(dists, descending=True)

#     plt.figure(figsize=(18,4))
#     plt.subplot(1, 6, 1)
#     plt.title(f'query:label:{category[sample.label]} idx{sample.idx}')
#     plt.imshow(sample.img)
    if(p==9):
        p=0
    for i, order in enumerate(orders[1:6]):
#         plt.subplot(1, 6, i+2)
        result = patch_train[order]
        train[sample.idx].si_score[p][train[result.idx].label]+=1
    p+=1
#         plt.title(f'mark:{dists[order]}')
#         plt.imshow(result.img)
#     if(idx>5):
#         break
p=0
for idx, sample in enumerate(patch_val):
    dists =  best_Net(sample.feat.view(1, -1).expand_as(all_feats), all_feats)
    simlarity, orders = torch.sort(dists, descending=True)

#     plt.figure(figsize=(18,4))
#     plt.subplot(1, 6, 1)
#     plt.title(f'query:label:{category[sample.label]} idx{sample.idx}')
#     plt.imshow(sample.img)
    if(p==9):
        p=0
    for i, order in enumerate(orders[1:6]):
#         plt.subplot(1, 6, i+2)
        result = patch_train[order]
        validation[sample.idx-len(train)].si_score[p][train[result.idx].label]+=1
    p+=1
#         plt.title(f'label{category[sample.label]}mark:{dists[order]}')
#         plt.imshow(result.img)
#     if(idx>5):
#         break

In [ ]:
validation[0].si_score.shape

In [ ]:
index=0
for sample in validation:
    real=0
    nostyle=0
    cyberpunk=0
    cartoon=0
    sample.pred=[]
    for patch in sample.si_score:
        patch_pred = np.argmax(patch)
        sample.pred.append(patch_pred)
    real=0
    nostyle=0
    cyberpunk=0
    cartoon=0
    for ppred in sample.pred:
        if (ppred==0):
            real+=1
        elif(ppred==1):
            cyberpunk+=1
        elif(ppred==2):
            cartoon+=1
        else:
            nostyle+=1
    if(real>=5):
        validation[index].pred=0
    else:
        if cyberpunk >= cartoon and cyberpunk >= nostyle:
            validation[index].pred=1
        elif cartoon >= cyberpunk and cartoon >= nostyle:
            validation[index].pred=2
        else:
            validation[index].pred=3
    index+=1


In [ ]:
pred=[sample.pred for sample in validation]
true=[sample.label for sample in validation]
accuracy = accuracy_score(pred, true)
print("Accuracy:", accuracy)

In [ ]:
pred

Directly using VGG feat

In [ ]:
train_feat=patch_train[0].feat
i=0
for sample in patch_train[1:]:
    train_feat=np.vstack((train_feat,sample.feat))
    
val_feat=patch_val[0].feat
for sample in patch_val[1:]:
    val_feat=np.vstack((val_feat,sample.feat))

test_feat=patch_test[0].feat
for sample in patch_test[1:]:
    test_feat=np.vstack((test_feat,sample.feat))

In [ ]:
train_label.shape

In [ ]:
sample_feat=np.concatenate((train_feat, val_feat), axis=0)

In [ ]:
sample_feat.shape

In [ ]:
# train_feat=[sample.feat for sample in patch_train]
train_label=np.array([sample.label for sample in patch_train])
# val_feat=[sample.feat for sample in patch_val]
val_label=np.array([sample.label for sample in patch_val]) 
# test_feat = [sample.feat for sample in patch_test]
test_label = np.array([sample.label for sample in patch_test])

In [ ]:
type(train_feat)

In [ ]:
all_feat=np.concatenate((train_feat, val_feat), axis=0)
all_label=np.concatenate((train_label,val_label), axis =0)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
# Set the number of folds
k = 10

# Create a KFold object
kf = KFold(n_splits=k, shuffle=True, random_state=69)
# classifier = SVC(kernel='linear')
accuracies=[]
best_accuracy=0
classifier = RandomForestClassifier()
# Iterate over the folds
for train_index, test_index in kf.split(all_feat):
    train_x, val_x = all_feat[train_index], all_feat[test_index]
    train_y, val_y = all_label[train_index], all_label[test_index]
    classifier = RandomForestClassifier()
    classifier.fit(train_x, train_y)

    predictions = classifier.predict(val_x)
#     print("Predictions:", predictions)
    accuracy = accuracy_score(val_y, predictions)
    print("Accuracy:", accuracy)
    if(accuracy>best_accuracy):
        best_accuracy=accuracy
        best_model=classifier
    accuracies.append(accuracy)

Stack classifier+voting classifier

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
# Define the individual classifiers
classifier_1 = LogisticRegression()
classifier_2 = RandomForestClassifier()
classifier_3 = SVC(probability=True)


# Set the number of folds
k = 10

# Create a KFold object
kf = KFold(n_splits=k, shuffle=True, random_state=69)
# classifier = SVC(kernel='linear')
accuracies=[]
best_accuracy=0
# Create the voting classifier
voting_classifier = VotingClassifier([('random_forest', classifier_2), ('svm', classifier_3)],
    voting='soft'  # Use majority voting
)

# Train the voting classifier using the training data
voting_classifier.fit(train_feat, train_label)

# Make predictions on the test data
predictions = np.argmax(voting_classifier.predict_proba(val_feat),axis=1)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(val_label, predictions)
print("Accuracy:", accuracy)
for train_index, test_index in kf.split(all_feat):
    train_x, val_x = all_feat[train_index], all_feat[test_index]
    train_y, val_y = all_label[train_index], all_label[test_index]

    # Train the voting classifier using the training data
    voting_classifier.fit(train_feat, train_label)

    # Make predictions on the test data
    predictions = np.argmax(voting_classifier.predict_proba(val_feat),axis=1)
    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(val_label, predictions)
    if(accuracy>best_accuracy):
        best_accuracy=accuracy
        best_model=voting_classifier
    accuracies.append(accuracy)
    print("Accuracy:", accuracy)

In [ ]:
best_model=voting_classifier

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list of values

# Generate x-axis values based on the index of each value in the list
x = range(len(accuracies))

# Plot the line chart
plt.plot(x, accuracies)

# Add labels and title
plt.xlabel('Fold')
plt.ylabel('Accuracy')
plt.title('K-fold Validation')

# Display the chart
plt.show()

In [ ]:
for sample in validation:
    patch=np.array([patch.numpy() for patch in sample.patches])
    sample.pred=np.argmax(best_model.predict_proba(patch),axis=1)
    print(sample.pred)
    real=0
    nostyle=0
    cyberpunk=0
    cartoon=0
    for ppred in sample.pred:
        if (ppred==0):
            real+=1
        elif(ppred==1):
            cyberpunk+=1
        elif(ppred==2):
            cartoon+=1
        else:
            nostyle+=1
    if(real>=5):
        sample.pred=0
    else:
        if cyberpunk >= cartoon and cyberpunk >= nostyle:
            sample.pred=1
        elif cartoon >= cyberpunk and cartoon >= nostyle:
            sample.pred=2
        else:
            sample.pred=3

Test part

In [ ]:
for sample in test:
    patch=np.array([patch.numpy() for patch in sample.patches])
    sample.pred=np.argmax(best_model.predict_proba(patch),axis=1)
    print(sample.pred)
    real=0
    nostyle=0
    cyberpunk=0
    cartoon=0
    for ppred in sample.pred:
        if (ppred==0):
            real+=1
        elif(ppred==1):
            cyberpunk+=1
        elif(ppred==2):
            cartoon+=1
        else:
            nostyle+=1
    if(real>=5):
        sample.pred=0
    else:
        if cyberpunk >= cartoon and cyberpunk >= nostyle:
            sample.pred=1
        elif cartoon >= cyberpunk and cartoon >= nostyle:
            sample.pred=2
        else:
            sample.pred=3

In [ ]:
pred=[sample.pred for sample in validation]
true=[sample.label for sample in validation]
accuracy = accuracy_score(pred, true)
print("Accuracy:", accuracy)

In [ ]:
pred=[sample.pred for sample in test]
for i in range(len(pred)):
    if(pred[i]!=0):
        pred[i]=3
true=[sample.label for sample in test]
accuracy = accuracy_score(pred, true)
print("Accuracy:", accuracy)

In [ ]:
len(true)

In [ ]:
mis_classified=[]
for i in range(len(pred)):
    if(pred[i]!=true[i]):
        mis_classified.append(Image.fromarray(test[i].img))

In [ ]:
len(mis_classified)

In [ ]:
fig, axes = plt.subplots(3, 3, figsize=(16, 16))
for i, ax in enumerate(axes.flat):
    if(i<5):
        ax.set_title(f'Gen-Nostyle')
    else:
        ax.set_title(f'Real')
    ax.imshow(mis_classified[i])
    ax.axis('off')

In [ ]:
Image.fromarray(test[5].img)

In [ ]:
import numpy as np

def patchify_image(image, patch_size):
    '''
    Patchify a colorful image into non-overlapping patches.

    Parameters:
        image (numpy.ndarray): Input image as a NumPy array.
        patch_size (tuple): Size of the patches (height, width).

    Returns:
        numpy.ndarray: Patchified image as a 4D array with shape (num_patches, patch_height, patch_width, num_channels).
    '''
    height, width, channels = image.shape
    patch_height, patch_width = patch_size

    num_patches_h = height // patch_height
    num_patches_w = width // patch_width
    num_patches = num_patches_h * num_patches_w

    patchified_image = np.zeros((num_patches, patch_height, patch_width, channels), dtype=image.dtype)

    index = 0
    for h in range(num_patches_h):
        for w in range(num_patches_w):
            patch = image[h*patch_height:(h+1)*patch_height, w*patch_width:(w+1)*patch_width, :]
            patchified_image[index] = patch
            index += 1

    return patchified_image

sample=validation[11]
# Load the colorful image
image = sample.img

# Define patch size and overlap
# Convert image to RGB if needed
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Define patch size
patch_size = (1008,1344)

# Patchify the image
patchified_image = patchify_image(image, patch_size)

# Display different patches
num_patches_to_show = 9

# Create a 4x4 subplot grid
fig, axes = plt.subplots(3, 3, figsize=(16, 16))

# Iterate over the patchified images and display them in the grid
for i, ax in enumerate(axes.flat):
    ax.set_title(f'{category[sample.pred[i]]}')
    ax.imshow(patchified_image[i])
    ax.axis('off')

# Adjust the spacing between subplots
plt.tight_layout()

# Show the grid of patchified images
plt.show()

In [ ]:
validation[11].pred